In [3]:
import tensorrt as trt
import numpy as np

print("🧪 Testing TensorRT functionality...")

# Тест базових функцій
logger = trt.Logger(trt.Logger.WARNING)
builder = trt.Builder(logger)
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

print("✅ All basic TensorRT components work!")
print("🚀 TensorRT is ready to use!")


ModuleNotFoundError: No module named 'tensorrt'

# Експорт YOLOv8n у ONNX (FP16)

У цьому ноутбуці ми:
- Встановимо потрібні бібліотеки (`ultralytics`, `onnx`, `onnxruntime`).
- Завантажимо ваги `yolov8n.pt` (завантажаться автоматично, якщо їх немає локально).
- Експортуємо модель у формат ONNX з напівточністю (FP16) і динамічними розмірами.
- Перевіримо експортований ONNX (структура та пробний запуск).

Примітки:
- FP16 ONNX краще виконувати на GPU/CPU з підтримкою FP16; на CPU можливі автоматичні перетворення типів.
- Шлях збереження можна змінити в змінній `save_dir`. За замовчуванням використаємо `trtexec_workspace/exports`.

In [ ]:
# Встановлення залежностей (запустіть один раз)

import sys, subprocess

pkgs = ["ultralytics>=8.3.0", "onnx", "onnxruntime"]

for p in pkgs:

    try:

        __import__(p.split("[")[0].split("<")[0].split(">")[0])

    except Exception:

        print(f"Installing {p}...")

        subprocess.check_call([sys.executable, "-m", "pip", "install", p])

print("✅ Dependencies ready")

In [ ]:
# Експорт YOLOv8n у ONNX FP16

from ultralytics import YOLO

import os



# Параметри експорту

imgsz = 640  # розмір інференсу (H=W)

opset = 17   # версія ONNX opset

save_dir = os.path.join("trtexec_workspace", "exports")

project = save_dir

name = "yolov8n_onnx_fp16"

os.makedirs(save_dir, exist_ok=True)



# Завантаження моделі (автоматично завантажить ваги, якщо їх немає локально)

model = YOLO("yolov8n.pt")



# Експорт у ONNX з напівточністю (FP16) та динамічними осями

onnx_path = model.export(

    format="onnx",

    opset=opset,

    half=True,        # FP16 ваги/обчислення, якщо підтримується бекендом

    dynamic=True,     # динамічні розміри (batch/height/width)

    simplify=True,    # спробувати спростити граф (потребує onnxsim)

    imgsz=imgsz,

    project=project,

    name=name,

    exist_ok=True,

)



print(f"✅ Експортовано: {onnx_path}")

In [ ]:
# Перевірка експортованого ONNX (структура та типи)

import onnx

import numpy as np

import onnxruntime as ort

from pathlib import Path



# Знайдемо останній експортований файл (Ultralytics повертає шлях або None)

search_dir = Path("trtexec_workspace/exports")

candidates = sorted(search_dir.rglob("*.onnx"))

assert candidates, "Не знайдено .onnx у trtexec_workspace/exports. Запустіть попередню клітинку."

onnx_file = str(candidates[-1])

print("ONNX файл:", onnx_file)



# 1) Перевірка валідності графа

model = onnx.load(onnx_file)

onnx.checker.check_model(model)

print("✅ ONNX пройшов перевірку onnx.checker")



# Порахувати кількість 16-бітних тензорів серед ініціалізаторів

fp16_inits = sum(1 for t in model.graph.initializer if t.data_type == onnx.TensorProto.FLOAT16)

fp32_inits = sum(1 for t in model.graph.initializer if t.data_type == onnx.TensorProto.FLOAT)

print(f"Ініціалізатори FP16: {fp16_inits}, FP32: {fp32_inits}")



# 2) Пробний інференс через ONNX Runtime (CPU)

sess = ort.InferenceSession(onnx_file, providers=["CPUExecutionProvider"])  # за потреби можна додати CUDA

input_name = sess.get_inputs()[0].name

print("Вхід:", sess.get_inputs()[0])

print("Виходи:", sess.get_outputs())



# Згенерувати випадковий вхід 1x3x640x640 у діапазоні [0,1]

dummy = np.random.rand(1, 3, 640, 640).astype(np.float32)  # runtime сам виконає приведення, якщо модель FP16

out = sess.run(None, {input_name: dummy})

print("Інференс успішний. К-сть виходів:", len(out))